In [2]:
import torch 
import torchvision
from torch.utils.data import DataLoader, Dataset
import os 
import sys
project_root = os.path.abspath("..")  # Adjust if needed

# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.mnist_vae import mnist_vae

In [6]:
def ELBO(x, x_hat,x_mu, x_logvar):

    rec_loss =  torch.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KL_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return (rec_loss + KL_loss) / x.size(0) 

In [5]:
latent_dim = 2
epochs = 100
lr = 0.01
batch_size = 20
x_dim = 28*28
if torch.cuda.is_available():
    torch.cuda.current_device()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = torchvision.transforms.ToTensor()

train_data = torchvision.datasets.MNIST('./datasets/', train=True, transform=transform,
                                        target_transform=None, download=True)
test_data = torchvision.datasets.MNIST('./datasets/', train=False, transform=transform,
                                       target_transform=None, download=True)


sample_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)

model = mnist_vae(latent_dim, device).to(device)
model_optim = torch.optim.Adam(params=model.parameters(), lr=lr)

In [12]:
model.train()
train_loss = []

for epoch in range(epochs):
        for batch_i, batch in enumerate(sample_dataloader):
                # forward pass
                x = batch[0].view(-1, x_dim).to(device)
                rep_z, x_mu, x_logvar, x_rec = model(x)
                loss = ELBO(x, x_rec,x_mu, x_logvar)
                batch_losses.append(loss.data.cpu().item())
                
                model_optim.zero_grad()
                loss.backward()
                model_optim.step()


                

torch.Size([20, 784])


In [9]:
next(iter(sample_dataloader))[1]

tensor([7, 2, 8, 6, 2, 5, 1, 2, 9, 9, 0, 0, 2, 5, 2, 4, 7, 9, 4, 4])